# Azure Synapse Linkを使用したAzure Cosmos DBコレクション全体の結合と集計
このノートで、あなたは、

1. [Azure Cosmos DB 分析ストア](https://docs.microsoft.com/ja-jp/azure/cosmos-db/analytical-store-introduction) コレクションを指す Spark テーブルを作成します
2. Perform Joins, filters and aggregations across collections using Spark-SQL
3. Visualize the data using plotly and displayHTML()

>**ご存知ですか？** [Azure Cosmos DB 用 Azure Synapse Link](https://docs.microsoft.com/ja-jp/azure/cosmos-db/synapse-link) は、Azure Cosmos DB の運用データに対してほぼリアルタイムの分析を実行できるハイブリッドトランザクション分析処理 (HTAP) 機能です。
&nbsp;

>**ご存知ですか？** [Azure Cosmos DB 分析ストア](https://docs.microsoft.com/ja-jp/azure/cosmos-db/analytical-store-introduction)は、トランザクションワークロードに影響を与えることなく、Azure Cosmos DB の運用データに対して大規模な分析を可能にする完全に分離された列ストアです。
&nbsp;

>**ご存知ですか？** Azure Cosmos DB コレクションの上に作成された Spark テーブルはメタデータテーブルであり、対応するコレクションにクエリを渡します。


### 1. Azure Synapse Link を使用して、Azure Cosmos DB 分析ストアコレクションを指す Spark テーブルを作成する
>**ご存知ですか？** 「cosmos.oltp」は、Cosmos DBトランザクションストアへの接続を可能にする Spark フォーマットです。

>**ご存知ですか？** Synapse の Cosmos DB Linked Service は、Azure Cosmos DB データベースの接続構成を抽象化します。

>リンクされたサービスをセットアップして Azure Cosmos DB データベースを Azure Synapse ワークスペースに接続する方法については、[こちらをクリック](https://docs.microsoft.com/ja-jp/azure/synapse-analytics/synapse-link/how-to-connect-synapse-link-cosmos-db#connect-an-azure-cosmos-db-database-to-a-synapse-workspace)してください。

In [3]:
%%sql
create database CosmosDBIoTDemo

In [4]:
%%sql

create table if not exists CosmosDBIoTDemo.IoTSignals
using cosmos.olap
options(spark.synapse.linkedService 'CosmosDBIoTDemo',
        spark.cosmos.container 'IoTSignals')


In [5]:
%%sql

create table if not exists CosmosDBIoTDemo.IoTDeviceInfo
using cosmos.olap
options(spark.synapse.linkedService 'CosmosDBIoTDemo',
        spark.cosmos.container 'IoTDeviceInfo')


### 2. コレクション全体で結合を実行し、Spark SQL を使用してフィルターと集計を適用する

>**ご存知ですか？** Cosmos DB 分析ストアに対するクエリは、運用データにプロビジョニングされたスループットを消費することなく、トランザクションワークロードトラフィックとは関係なく提供されます。

>**ご存知ですか？** Cosmos DB 分析ストアは、列型ストレージを使用し、大規模な分析ワークロード用に最適化されています。

>**ご存知ですか？** Cosmos DB 分析ストアは、データストレージおよび分析の読み取り/書き込み操作と実行されたクエリに基づいており、スループットをプロビジョニングする必要がない、費用対効果の高い消費ベースの料金モデルに従います。

In [6]:
df_RPM_details = spark.sql("select a.deviceid \
                                 , b.devicetype \
                                 , cast(b.location as string) as location\
                                 , cast(b.latitude as float) as latitude\
                                 , cast(b.longitude as float) as  longitude\
                                 , a.measuretype \
                                 , a.unitSymbol \
                                 , cast(sum(measureValue) as float) as measureValueSum \
                                 , count(*) as count \
                            from CosmosDBIoTDemo.IoTSignals a \
                            left join CosmosDBIoTDemo.IoTDeviceInfo b \
                            on a.deviceid = b.deviceid \
                            where a.unitSymbol = 'RPM' \
                            group by a.deviceid, b.devicetype, b.location, b.latitude, b.longitude, a.measuretype, a.unitSymbol")

### display() 関数を使用して結果セットを表示またはグラフ化する


In [7]:
display(df_RPM_details)

### 3. plotly と displayHTML() を使用して可視化する
以下は、さまざまな場所にわたる IoT 信号のヒートマップを示しています。

In [8]:
from plotly.offline import plot
import plotly.express as px

df_RPM_details_pd = df_RPM_details.toPandas()
fig = px.scatter_mapbox(df_RPM_details_pd, 
                        lat='latitude', 
                        lon='longitude', 
                        size = 'measureValueSum',
                        color = 'measureValueSum',
                        hover_name = 'location',
                        hover_data = ['measureValueSum','location'],
                        size_max = 30,
                        color_continuous_scale = px.colors.carto.Temps,
                        zoom=3,
                        height=600,
                        width =900)

fig.update_layout(mapbox_style='open-street-map')
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

p = plot(fig,output_type='div')
displayHTML(p)       

<img src="https://revin.blob.core.windows.net/synapselinknotebooks/RPMHeatmap.PNG" width="1400" style="float: center;"/>
